In [ ]:
from openevals.code.e2b.pyright import create_e2b_pyright_evaluator
from openevals.code.e2b.execution import create_e2b_execution_evaluator
from e2b_code_interpreter import Sandbox
from dotenv import load_dotenv

load_dotenv()
sandbox = Sandbox('OpenEvalsPython')

evaluator = create_e2b_pyright_evaluator(
    sandbox=sandbox,
)

CODE = """
from typing import Annotated, TypedDict
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()
model = ChatOpenAI ()
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chat_node(state: State): 
    messages = state['messages']

    response = model.invoke(messages).content 

    return {
        'messages': [response]
    }


builder = StateGraph(State)
builder.add_node('chat', chat_node)

builder.add_edge(START, 'chat')
builder.add_edge('chat', END)
graph = builder.compilee()

result = graph.invoke({'messages': [HumanMessage('How are you?')]})

print(result)

"""

eval_result = evaluator(outputs=CODE)

print(eval_result)

{'key': 'pyright_succeeded', 'score': False, 'comment': '[{"severity": "error", "message": "Cannot access attribute \\"compilee\\" for class \\"StateGraph[State, None, State, State]\\"\\n\\u00a0\\u00a0Attribute \\"compilee\\" is unknown", "range": {"start": {"line": 28, "character": 16}, "end": {"line": 28, "character": 24}}, "rule": "reportAttributeAccessIssue"}]', 'metadata': None}


In [7]:
for k,v in eval_result.items(): 
  print(k)
  print(v)
  print()

key
pyright_succeeded

score
False

comment
[{"severity": "error", "message": "Cannot access attribute \"invoke\" for class \"StateGraph[State, None, State, State]\"\n\u00a0\u00a0Attribute \"invoke\" is unknown", "range": {"start": {"line": 16, "character": 8}, "end": {"line": 16, "character": 14}}, "rule": "reportAttributeAccessIssue"}]

metadata
None



In [24]:
eval_result['comment'].split(', "')

['[{"severity": "error"',
 'message": "Cannot access attribute \\"compilee\\" for class \\"StateGraph[State, None, State, State]\\"\\n\\u00a0\\u00a0Attribute \\"compilee\\" is unknown"',
 'range": {"start": {"line": 28',
 'character": 16}',
 'end": {"line": 28',
 'character": 24}}',
 'rule": "reportAttributeAccessIssue"}]']

In [12]:
import ast
from pprint import pprint
tree = ast.parse(CODE)

pprint(ast.dump(tree))


("Module(body=[ImportFrom(module='typing', names=[alias(name='Annotated'), "
 "alias(name='TypedDict')], level=0), ImportFrom(module='dotenv', "
 "names=[alias(name='load_dotenv')], level=0), "
 "ImportFrom(module='langgraph.graph', names=[alias(name='StateGraph'), "
 "alias(name='START'), alias(name='END')], level=0), "
 "ImportFrom(module='langgraph.graph.message', "
 "names=[alias(name='add_messages')], level=0), "
 "ImportFrom(module='langchain_core.messages', "
 "names=[alias(name='HumanMessage')], level=0), "
 "ImportFrom(module='langchain_openai', names=[alias(name='ChatOpenAI')], "
 "level=0), Expr(value=Call(func=Name(id='load_dotenv', ctx=Load()), args=[], "
 "keywords=[])), Assign(targets=[Name(id='model', ctx=Store())], "
 "value=Call(func=Name(id='ChatOpenAI', ctx=Load()), args=[], keywords=[])), "
 "ClassDef(name='State', bases=[Name(id='TypedDict', ctx=Load())], "
 "keywords=[], body=[AnnAssign(target=Name(id='messages', ctx=Store()), "
 "annotation=Subscript(value=Name(

In [14]:
type(ast.dump(tree))

str

In [ ]:
from ast import Import 
node = ast.parse(CODE)
tree = ast.dump(tree)

tree.Import 

# walk thru the tree 
# if we encounter alias, add it to imports 





TypeError: expected AST, got 'str'

In [34]:
import sys
def _is_stdlib(pkg: str) -> bool:
        """Detect if a top-level import is from the stdlib (best-effort)."""
        # Python 3.10+: sys.stdlib_module_names
        stdmods = getattr(sys, "stdlib_module_names", None)
        if stdmods is not None:
            return pkg in stdmods

In [39]:
_is_stdlib('langgraph')

False

In [ ]:
from typing import Dict, List, Any, Set
import ast 
import sys 

def _collect_imports_ast(code: str) -> Set[str]: 
  try: 
    tree = ast.parse(code)
  except SyntaxError: 
    return set()
  
  packages: Set[str] = set()

  class ImportVisitor(ast.NodeVisitor): 
    def visit_Import(self, node: ast.Import): 
      for alias in node.names: 
        packages.add(alias.name.split('.')[0])
      self.generic_visit(node)


    def visit_ImportFrom(self, node: ast.ImportFrom): 
      print(node.module)
      if node.module: 
        packages.add(node.module.split('.')[0]) 
      self.generic_visit(node)

  ImportVisitor().visit(tree)
  return packages

code = """ 
import os
import sys
import math
import json
import time
import random
import string
import re
import uuid
import base64
import hashlib
import logging
import threading
import asyncio
import concurrent.futures
import statistics
import functools
import itertools
import tempfile
import shutil
import pathlib
import datetime
import collections
import inspect
import socket
import http.client
import urllib.request
import urllib.parse
import zipfile
import tarfile
import mimetypes
import csv
import sqlite3
import subprocess
import platform
import ctypes
import signal
import typing
import pprint

# Third-party imports (these would require pip install)
try:
    import requests
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import bs4
    from bs4 import BeautifulSoup
    import yaml
    import pytz
    import dateutil
    from PIL import Image
    import flask
    from flask import Flask, jsonify
    import fastapi
    from fastapi import FastAPI
    import sqlalchemy
    import sklearn
    import tensorflow as tf
    import torch
    import plotly.express as px
except ImportError:
    print("Some third-party packages are not installed. Install them with pip if needed.")

# Random functional chaos
def random_string(length=8):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

def compute_hash(data: str):
    return hashlib.sha256(data.encode()).hexdigest()

def async_print_numbers():
    async def inner():
        for i in range(5):
            await asyncio.sleep(0.5)
            print(f"Number: {i}")
    return asyncio.run(inner())

def random_dataframe():
    df = pd.DataFrame({
        'id': [uuid.uuid4() for _ in range(5)],
        'value': np.random.rand(5),
        'timestamp': pd.date_range(start="2025-01-01", periods=5, freq="D")
    })
    print(df)
    return df

def scrape_example():
    url = "https://example.com"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    print("Title:", soup.title.string if soup.title else "No title")
"""

imports = _collect_imports_ast(code)
print(imports)

bs4
PIL
flask
fastapi
{'dateutil', 'functools', 'statistics', 'threading', 'sqlalchemy', 'sklearn', 'tempfile', 'zipfile', 'signal', 'pytz', 'tarfile', 'yaml', 'PIL', 're', 'tensorflow', 'ctypes', 'matplotlib', 'fastapi', 'inspect', 'datetime', 'typing', 'itertools', 'socket', 'numpy', 'random', 'requests', 'urllib', 'string', 'hashlib', 'shutil', 'os', 'concurrent', 'pprint', 'torch', 'bs4', 'math', 'asyncio', 'sqlite3', 'pandas', 'sys', 'flask', 'uuid', 'seaborn', 'platform', 'mimetypes', 'csv', 'pathlib', 'json', 'time', 'collections', 'subprocess', 'base64', 'logging', 'plotly', 'http'}


In [72]:
imports = ast.dump(ast.parse("""
import x as ex,z as zee
import y as yolo
from a import b
from a.m import c as cee, d, f
"""))

In [73]:
import pprint
pprint.pprint(imports) 

("Module(body=[Import(names=[alias(name='x', asname='ex'), alias(name='z', "
 "asname='zee')]), Import(names=[alias(name='y', asname='yolo')]), "
 "ImportFrom(module='a', names=[alias(name='b')], level=0), "
 "ImportFrom(module='a.m', names=[alias(name='c', asname='cee'), "
 "alias(name='d'), alias(name='f')], level=0)], type_ignores=[])")


In [76]:

def _is_stdlib(pkg: str) -> bool:
  """Detect if a top-level import is from the stdlib (best-effort)."""
  stdmods = getattr(sys, "stdlib_module_names", None)
  if stdmods is not None:
      return pkg in stdmods
  


In [78]:
_is_stdlib('pandas')

False

In [ ]:
from openevals.code.pyright import create_pyright_evaluator
def _validate_syntax(code: str) -> Dict[str, Any]: 
  evaluator = create_pyright_evaluator()
  result = evaluator(outputs=code)
  return result

In [89]:
results = _validate_syntax(CODE)
print(results)

{'key': 'pyright_succeeded', 'score': False, 'comment': '[{"severity": "error", "message": "Cannot access attribute \\"compilee\\" for class \\"StateGraph[State, State, State]\\"\\n\\u00a0\\u00a0Attribute \\"compilee\\" is unknown", "range": {"start": {"line": 28, "character": 16}, "end": {"line": 28, "character": 24}}, "rule": "reportAttributeAccessIssue"}]', 'metadata': None}


In [93]:
results.get('comment')

'[{"severity": "error", "message": "Cannot access attribute \\"compilee\\" for class \\"StateGraph[State, State, State]\\"\\n\\u00a0\\u00a0Attribute \\"compilee\\" is unknown", "range": {"start": {"line": 28, "character": 16}, "end": {"line": 28, "character": 24}}, "rule": "reportAttributeAccessIssue"}]'

In [133]:
ast.literal_eval(results.get('comment'))

[{'severity': 'error',
  'message': 'Cannot access attribute "compilee" for class "StateGraph[State, State, State]"\n\xa0\xa0Attribute "compilee" is unknown',
  'range': {'start': {'line': 28, 'character': 16},
   'end': {'line': 28, 'character': 24}},
  'rule': 'reportAttributeAccessIssue'}]

In [142]:
code_new = """ 
a = 10
b = 23
print(a+b/0)
"""

In [169]:
code_new2 = """ 
a = 10
b = 23
printt(a+b)
"""

In [139]:
results_new

{'key': 'execution_succeeded',
 'score': True,
 'comment': None,
 'metadata': None}

In [143]:
def _run_in_sandbox(code: str) -> Dict[str, Any]: 
    try: 
      with Sandbox('OpenEvalsPython') as sandbox: 
        evaluator = create_e2b_execution_evaluator(
          sandbox=sandbox
        )
        eval_result = evaluator(outputs=code)
        return {
          'success': eval_result['score'], 
          'errors': None if not eval_result['comment'] else eval_result['comment']
          }
    except Exception as e: 
      print('Exception block executed')
      return {
        'success': False, 
        'errors': str(e),
      } 
    
exe_results = _run_in_sandbox(code_new)

In [149]:
exe_results

{'success': False,
 'errors': 'Command exited with code 1 and error:\nTraceback (most recent call last):\n  File "/home/user/openevals/outputs.py", line 4, in <module>\n    print(a+b/0)\nZeroDivisionError: division by zero\n'}

In [151]:
result2 = _run_in_sandbox(code_new2)

In [156]:
a = result2.get('errors')

In [160]:
type(result2.get('success'))

bool

In [174]:
def _validate_syntax(code: str) -> Dict[str, Any]: 
  evaluator = create_pyright_evaluator()
  result = evaluator(outputs=code)
  return result  


In [187]:
syntax_check = _validate_syntax(code_new)

In [188]:
syntax_check

{'key': 'pyright_succeeded', 'score': True, 'comment': '[]', 'metadata': None}

In [ ]:
comment = syntax_check.get('comment')
comment = ast.literal_eval(comment)

In [ ]:
comment[0]

{'severity': 'error',
 'message': '"printt" is not defined',
 'range': {'start': {'line': 3, 'character': 0},
  'end': {'line': 3, 'character': 6}},
 'rule': 'reportUndefinedVariable'}